# LangChain と PaLM API で PDF ドキュメントの要約を行う例

LangChain のパッケージ、および、PDF の扱いに必要なパッケージをインストールします。

In [ ]:
!pip install --user \
  langchain==0.1.0 transformers==4.36.0 \
  pypdf==3.17.0 cryptography==42.0.4 \
  langchain-google-vertexai==0.0.5 \
  google-cloud-aiplatform==1.39.0

**注意：次のセルを実行する前にカーネルをリスタートしてください。**

NISC が一般公開している「[インターネットの安全・安心ハンドブック](https://security-portal.nisc.go.jp/guidance/handbook.html)」のプロローグ部分をダウンロードします。





In [1]:
base_url = 'https://raw.githubusercontent.com/google-cloud-japan/sa-ml-workshop/main'
!wget -q $base_url/genAI_book/PDF/handbook-prologue.pdf

ダウンロードした PDF を読み込んで、全体を1つのテキストにまとめます。

In [2]:
from langchain_community.document_loaders import PyPDFLoader

pages = PyPDFLoader('handbook-prologue.pdf').load()
document = ''
for page in pages:
    document += page.page_content

PaLM API を利用するクライアントオブジェクトを取得します。

In [3]:
from langchain_google_vertexai import VertexAI
llm = VertexAI(model_name='text-bison@002',
               temperature=0.1, max_output_tokens=1024)

RecursiveCharacterTextSplitter, load_qa_chain, AnalyzeDocumentChain モジュールの使用例です。

`input_document` に与えたテキストに基づいて、`question` の質問に回答します。

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=6000, chunk_overlap=200)
qa_chain = load_qa_chain(llm, chain_type='map_reduce')
qa_document_chain = AnalyzeDocumentChain(
    combine_docs_chain=qa_chain, text_splitter=text_splitter)

output = qa_document_chain.invoke(
    {'input_document':'今は６月で雨が多い時期です。', 'question':'最近の天候は？'})
print(output)

{'input_document': '今は６月で雨が多い時期です。', 'question': '最近の天候は？', 'output_text': ' 最近の天候は雨が多いです。'}


`input_document` に与えたテキストの内容をまとめるように指示して結果を得る関数を用意します。

In [5]:
def get_description(document):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, chunk_overlap=200)
    qa_chain = load_qa_chain(llm, chain_type='map_reduce')
    qa_document_chain = AnalyzeDocumentChain(
        combine_docs_chain=qa_chain, text_splitter=text_splitter)

    prompt = '何についての文書ですか？日本語で200字程度にまとめて教えてください。'
    description = qa_document_chain.invoke(
        {'input_document': document, 'question': prompt})
    return description['output_text']

先に用意した PDF ファイルを適用します。

In [9]:
print(get_description(document))

 この文書は、インターネットを利用する際に注意すべきリスクやトラブル、およびそれらへの対策について解説しています。


任意の質問に答える関数を用意します。

In [7]:
def get_answer(document, question):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, chunk_overlap=200)
    qa_chain = load_qa_chain(llm, chain_type='refine')
    qa_document_chain = AnalyzeDocumentChain(
        combine_docs_chain=qa_chain, text_splitter=text_splitter)

    prompt = '{} 日本語で200字程度にまとめて教えてください。'.format(question)
    answer = qa_document_chain.invoke(
        {'input_document': document, 'question': prompt})
    return answer['output_text']

先に用意した PDF ファイルに基づいて、質問の回答を取得します。

In [13]:
question = 'サイバーセキュリティ対策のポイントを箇条書きにまとめてください。'
print(get_answer(document, question))

 サイバーセキュリティ対策のポイント

・OSやソフトウェアは常に最新の状態にしておく。
・多要素認証を利用する。
・偽メールや偽サイトに騙されないように用心する。
・スマホやPCの画面ロックを利用する。
・大切な情報は失う前にバックアップする。
・外出先では紛失・盗難・覗き見に注意する。
・困ったときは1人で悩まず、まず相談する。
・パスワードは長く複雑にして、他と使い回さないようにする。
・心当たりのない送信元からのメールに添付されているファイルやリンクには注意する。
